In [1]:
# Import Librairies

import json 
import numpy as np 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

In [2]:
# Read Json file

with open('intents.json') as file:
    data = json.load(file)
    
training_sentences = []
training_labels = []
labels = []
responses = []


for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])
    
    if intent['tag'] not in labels:
        labels.append(intent['tag'])
        
num_classes = len(labels)

print(data['intents'])

[{'tag': 'greeting', 'patterns': ['Hi', 'Hey', 'Is anyone there?', 'Hello', 'Hay', 'Wassup bro', 'A fin', 'Wech kho'], 'responses': ['Hello', 'Hi', 'Hi there', 'Hi there']}, {'tag': 'goodbye', 'patterns': ['Bye', 'See you later', 'Goodbye', 'Ciao', 'Tchuss', 'Thella'], 'responses': ['See you later', 'Have a nice day', 'Bye! Come back again', 'Auf Wiedersehen']}, {'tag': 'thanks', 'patterns': ['Thanks', 'Thank you', "That's helpful", 'Thanks for the help', 'la7afdek', 'Itoub'], 'responses': ['Happy to help!', 'Any time!', 'My pleasure', "You're most welcome!", 'Hania a khouya']}, {'tag': 'about', 'patterns': ['Who are you?', 'What are you?', 'Who you are?', 'Chkoun nta'], 'responses': ['I.m NawSEG, your bot assistant', "I'm NawSEG, an A.I bot", 'Je suis ton Pere ... Starwars']}, {'tag': 'name', 'patterns': ['what is your name', 'what should I call you', 'whats your name?'], 'responses': ['You can call me NawSEG.', "I'm NawSEG!", 'Just call me as NawSEG']}, {'tag': 'help', 'patterns': ['

In [3]:
# Label Encoder Method is encoding the levels of categorical features into numeric values
# LabelEncoder encode labels with a value between 0 and n_classes-1 where n is the number of distinct labels (here n )

lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels = lbl_encoder.transform(training_labels)

training_labels

array([4, 4, 4, 4, 4, 4, 4, 4, 3, 3, 3, 3, 3, 3, 7, 7, 7, 7, 7, 7, 0, 0,
       0, 0, 6, 6, 6, 5, 5, 5, 5, 5, 5, 5, 2, 2, 2, 2, 2, 1, 1, 1])

In [4]:
# Tokenization process (Vectorising the Data)

vocab_size = 1000 # the size of the vocabulary in the text data
embedding_dim = 16
max_len = 20
oov_token = "<OOV>" # Out Of Vocabulary are words not present in the vocabulary and will be mapped as UNK token

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len) # 'pre' or 'post' (optional, defaults to 'pre'): remove values from sequences larger than maxlen, either at the beginning or at the end of the sequences.

In [5]:
# Training the Neural Network of our Chatbot with 3 Hidden layers

model = Sequential() # The sequential API allows you to create models layer-by-layer for most problems. It is limited in that it does not allow you to create models that share layers or have multiple inputs or outputs.
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D()) # Pooling. Its function is to progressively reduce the spatial size of the representation to reduce the amount of parameters and computation in the network
model.add(Dense(16, activation='relu')) # Activation function Relu 1
model.add(Dense(16, activation='relu')) # Activation function Relu 2
model.add(Dense(num_classes, activation='softmax')) # Activation function Softmax

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy']) # This loss function is mathematically same as the categorical_crossentropy.

model.summary()
epochs = 500
history = model.fit(padded_sequences, np.array(training_labels), epochs=epochs)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 16)            16000     
                                                                 
 global_average_pooling1d (G  (None, 16)               0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 16)                272       
                                                                 
 dense_1 (Dense)             (None, 16)                272       
                                                                 
 dense_2 (Dense)             (None, 8)                 136       
                                                                 
Total params: 16,680
Trainable params: 16,680
Non-trainable params: 0
____________________________________________________

In [6]:
# to save the trained model for any future use
model.save("chat_model")

import pickle

# to save the fitted tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# to save the fitted label encoder
with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)
    
# Pickle in Python is primarily used in serializing and deserializing a Python object structure. 
# In other words, it's the process of converting a Python object into a byte stream to store it in a file/database, maintain program state across sessions, or transport data over the network.

Assets written to: chat_model\assets
